In [1]:
# Import useful packages
import os
import time
import pandas as pd
# Suppressing some warnings in pandas
pd.options.mode.chained_assignment = None
import numpy as np
import glob
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, date, time, timedelta
from ta import add_all_ta_features

# Importing our self-created functions
from feature_creation import *
from portfolio import *

# Needed to scrap the web
import requests 
from bs4 import BeautifulSoup

# Importing for plotting
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import dash_table
import dash_bootstrap_components as dbc

In [2]:
# Create stock_df to get tickers
stock_df = daily_features()

stock_df.head()

,Adj Close,Close,Close_adj,Dividends,High,High_adj,Low,Low_adj,Open,Open_adj,Stock Splits,Volume,sector,ticker,SMA_5,SMA_15,SMA_ratio,SD,upperband,lowerband
Date,,,,,,,,,,,,,,,,,,,,
1990-02-16,0.056967,0.056967,0.077257,0.0,0.058887,0.079861,0.054407,0.073785,0.0,0.0,0.0,940636800.0,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN
1990-02-20,0.058887,0.058887,0.079861,0.0,0.058887,0.079861,0.055047,0.074653,0.0,0.0,0.0,151862400.0,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN
1990-02-21,0.057607,0.057607,0.078125,0.0,0.058247,0.078993,0.055687,0.075521,0.0,0.0,0.0,70531200.0,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN
1990-02-22,0.058247,0.058247,0.078993,0.0,0.060167,0.081597,0.058247,0.078993,0.0,0.0,0.0,45216000.0,Technology,CSCO,NaN,NaN,NaN,NaN,NaN,NaN
1990-02-23,0.057927,0.057927,0.078559,0.0,0.058887,0.079861,0.057607,0.078125,0.0,0.0,0.0,44697600.0,Technology,CSCO,0.057927,NaN,NaN,NaN,NaN,NaN


In [102]:
stock_df[stock_df['ticker']=='CSCO']['sector'].unique()[0]

'Technology'

In [3]:
# Creating list of tickers to scrap sentiment for (should be entire S&P 500)
tickers = list(stock_df.ticker.unique())

In [4]:
# Initializing scraping capabilities for sentdex on last 30 day sentiment
res = requests.get('http://www.sentdex.com/financial-analysis/?tf=30d')
soup = BeautifulSoup(res.text)
table = soup.find_all('tr')

In [5]:
# Initialize empty lists to store stock symbol, sentiment and mentions
stock = []
sentiment = []
mentions = []
sentiment_trend = []

In [6]:
# Use try and except blocks to mitigate missing data
for ticker in table:
    ticker_info = ticker.find_all('td')
    
    try:
        stock.append(ticker_info[0].get_text())
    except:
        stock.append(None)
    try:
        sentiment.append(ticker_info[3].get_text())
    except:
        sentiment.append(None)
    try:
        mentions.append(ticker_info[2].get_text())
    except:
        mentions.append(None)
    try:
        if (ticker_info[4].find('span',{"class":"glyphicon  glyphicon-chevron-up"})):
            sentiment_trend.append('up')
        else:
            sentiment_trend.append('down')
    except:
        sentiment_trend.append(None)

In [124]:
sentiment_df = pd.DataFrame(
    data={'Ticker': stock,
     'Sentiment': sentiment,
     'Mentions': mentions,
     'Trend': sentiment_trend,
    },
    dtype=str)

In [125]:
sentiment_df.head()

,Ticker,Sentiment,Mentions,Trend
0,None,None,None,None
1,SP500,very good,124814,down
2,NOK,very good,8996,down
3,SNE,good,1616,down
4,MAT,very good,1309,down


In [127]:
tick_sector_df = stock_df.groupby(['ticker', 'sector']).count().reset_index()[['ticker','sector']]

In [128]:
# Adding sector to sentiment_df for filtering purposes
sentiment_df2 = sentiment_df.merge(tick_sector_df,how='left',left_on='Ticker',right_on='ticker')
sentiment_df2 = sentiment_df2[sentiment_df2['ticker'].notna()]

sentiment_df2['Mentions'] = sentiment_df2['Mentions'].astype('int')

sentiment_df2.head()

,Ticker,Sentiment,Mentions,Trend,ticker,sector
5,LEG,very good,761,down,LEG,Consumer Cyclical
6,K,very good,659,down,K,Consumer Defensive
8,AIG,good,549,down,AIG,Financial Services
12,NWSA,good,430,down,NWSA,Communication Services
13,HBAN,good,416,down,HBAN,Financial Services


In [120]:
# Looks like we are missing quite a few big tickers...
tick_sector_df[tick_sector_df['ticker']=='SNE']

,ticker,sector


In [129]:
# Saving data to CSV for loading in dashboard
sentiment_df2.to_csv('assets/models/tyler_rf_daily_update/sentiment_analysis.csv',index=False)

In [131]:
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.SUPERHERO])

df = sentiment_df2[sentiment_df2['sector']=='Financial Services'].nlargest(10,'Mentions')[['Ticker','Sentiment','Trend','Mentions']]

app.layout = html.Div([
    dash_table.DataTable(
        id='sentiment_table',
        data=df.to_dict('records'),
        columns=[{"name": i, "id": i} for i in df.columns],
        style_cell=dict(textAlign='left'),
        style_header=dict(backgroundColor="#191970",color='white'),
        style_data=dict(backgroundColor="gray",color='black'),
        
        # Setting conditional styles for sentiment and trends
         style_data_conditional=[
            {'if': {'filter_query': '{Sentiment} = "good" || {Sentiment} = "very good"','column_id': 'Sentiment'},
                'color': '#03CD1E'},
            {'if': {'filter_query': '{Sentiment_Trend} = "down"','column_id': 'Sentiment_Trend'},
                'color': '#FF0000'},
            {'if': {'filter_query': '{Sentiment_Trend} = "up"','column_id': 'Sentiment_Trend'},
                'color': '#03CD1E'}],
        
        style_as_list_view=True
    )
])

app.run_server(mode='inline',port=8090)

---------------------------------------------------------------------------
NotFound                                  Traceback (most recent call last)
~/Documents/Grad School/SIADS697/venv/lib/python3.7/site-packages/flask/app.py in full_dispatch_request(self=<Flask '__main__'>)
   1511             rv = self.preprocess_request()
   1512             if rv is None:
-> 1513                 rv = self.dispatch_request()
        rv = None
        self.dispatch_request = <bound method Flask.dispatch_request of <Flask '__main__'>>
   1514         except Exception as e:
   1515             rv = self.handle_user_exception(e)

~/Documents/Grad School/SIADS697/venv/lib/python3.7/site-packages/flask/app.py in dispatch_request(self=<Flask '__main__'>)
   1497             return self.make_default_options_response()
   1498         # otherwise dispatch to the handler for that endpoint
-> 1499         return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
        self.ensure_sync